### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

#Create columns to indicate whether students have passing grades and add to student data frame
student_data["passing_reading"] = student_data["reading_score"] >= 70
student_data["passing_math"] = student_data["math_score"] >= 70
student_data["overall_passing"] = (student_data["reading_score"] >= 70) & (student_data["math_score"] >= 70)

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Count Schools
school_count = school_data["school_name"].count()

#Sum students
student_count = school_data["size"].sum()

#Sum budget
district_budget = school_data["budget"].sum()

#Average all math scores
district_math_avg = student_data["math_score"].mean()

#Average all reading scores
district_reading_avg = student_data["reading_score"].mean()

#Calculate percent passing math
passing_math_perc = (np.sum(student_data["passing_math"] == True) / student_count) * 100

#Calculate percent passing reading

passing_reading_perc = (np.sum(student_data["passing_reading"] == True) / student_count) * 100

#Calculate passing overall

passing_overall_perc = (np.sum(student_data["overall_passing"] == True) / student_count) * 100

#Pass all summary values above into a data frame
district_summary_df = pd.DataFrame({
    "School_Count": [school_count],
    "Student_Count": student_count,
    "District_Budget": district_budget,
    "District_Math_Average": district_math_avg,
    "District_Reading_Average": district_reading_avg,
    "Students_Passing_Math": passing_math_perc,
    "Students_Passing_Reading": passing_reading_perc,
    "Students_Passing": passing_overall_perc
})

#Mapping to clen up column formatting
district_summary_df["Student_Count"] = district_summary_df["Student_Count"].map("{:,}".format)
district_summary_df["District_Budget"] = district_summary_df["District_Budget"].map("${:,}".format)
district_summary_df["District_Math_Average"] = district_summary_df["District_Math_Average"].map("{:.2f}".format)
district_summary_df["District_Reading_Average"] = district_summary_df["District_Reading_Average"].map("{:.2f}".format)
district_summary_df["Students_Passing_Math"] = district_summary_df["Students_Passing_Math"].map("%{:.2f}".format)
district_summary_df["Students_Passing_Reading"] = district_summary_df["Students_Passing_Reading"].map("%{:.2f}".format)
district_summary_df["Students_Passing"] = district_summary_df["Students_Passing"].map("%{:.2f}".format)

#Rename columns
clean_district_summary = district_summary_df.rename(columns={
    "School_Count": "Number of Schools",
    "Student_Count": "Student Population",
    "District_Budget": "District Budget",
    "District_Math_Average": "District-Wide Math Average",
    "District_Reading_Average": "District-Wide Reading Average",
    "Students_Passing_Math": "% of Students Passing Math",
    "Students_Passing_Reading": "% of Students Passing Reading",
    "Students_Passing": "% of Students Passing"
})



clean_district_summary



,Number of Schools,Student Population,District Budget,District-Wide Math Average,District-Wide Reading Average,% of Students Passing Math,% of Students Passing Reading,% of Students Passing
0,15,"39,170","$24,649,428",78.99,81.88,%74.98,%85.81,%65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [18]:
#Pull data that does not need to be changed into a new data frame to start
school_summary_df = school_data[["school_name", "type", "size", "budget"]]

#Aphabetize schools (important when adding in groupby statements)
school_summary_df.sort_values("school_name").reset_index(drop=True)

#Caluclate budget per student and add to data frame
school_summary_df["budget_per_student"] = school_summary_df["budget"] / school_summary_df["size"]


#Create a groupby object by school to pull out scores
groupby_school = student_data.groupby(["school_name"])


#Use groupby school object to average test scores
avg_math_by_school = groupby_school["math_score"].mean()
avg_reading_by_school = groupby_school["reading_score"].mean()

#Place scores in a data frame
test_avgs_df = pd.DataFrame({"avg_math": avg_math_by_school, "avg_reading": avg_reading_by_school})

#Join partial summary table with socres table on school name
summary_with_scores = pd.merge(school_summary_df, test_avgs_df, on="school_name")

#Transition to percent work
#filter entire student data table for only those that passed math
passed_math = student_data.loc[student_data["passing_math"] == True, :]

#Group by school and count the number of students who passed
math_pass_by_school = passed_math.groupby(["school_name"]).count()

#repeat the above two steps for reading scores
passed_reading = student_data.loc[student_data["passing_reading"] == True, :]

reading_pass_by_school = passed_reading.groupby(["school_name"]).count()

#do the same for overall scores
passed_overall = student_data.loc[student_data["overall_passing"] == True, :]

overall_pass_by_school = passed_overall.groupby(["school_name"]).count()

#Combine the above three count series into a data frame
passing_df = pd.DataFrame({"passing_math_count": math_pass_by_school["passing_math"], 
                          "passing_reading_count": reading_pass_by_school["passing_reading"], 
                          "overall_passing_count": overall_pass_by_school["overall_passing"]})


#index with school so that I can divide the columns
indexed_school_summary_df = school_summary_df.set_index(school_summary_df["school_name"])

#create columns to show percent passing by dividing each count by school size and inserting into the data frame
passing_df["passing_math_perc"] = (passing_df["passing_math_count"] / indexed_school_summary_df["size"]) * 100

passing_df["passing_reading_perc"] = (passing_df["passing_reading_count"] / indexed_school_summary_df["size"]) * 100

passing_df["passing_overall_perc"] = (passing_df["overall_passing_count"] / indexed_school_summary_df["size"]) * 100

#Join half-completed summary from above to the percent passing df
full_summary_df = pd.merge(summary_with_scores, passing_df, on="school_name")

#Delete columns for passing counts
del full_summary_df["passing_math_count"]
del full_summary_df["passing_reading_count"]
del full_summary_df["overall_passing_count"]

sorted_full_summary = full_summary_df.sort_values("school_name").reset_index(drop=True)

#Assign data to new data frame so I can format it without messing with the sorted_full_suarry, 
#which is referenced in code below

mapped_full_summary = sorted_full_summary

#map columns for formatting
mapped_full_summary["size"] = mapped_full_summary["size"].map("{:,}".format)
mapped_full_summary["budget"] = mapped_full_summary["budget"].map("${:,}".format)
mapped_full_summary["budget_per_student"] = mapped_full_summary["budget_per_student"].map("${:.2f}".format)
mapped_full_summary["avg_math"] = mapped_full_summary["avg_math"].map("{:.2f}".format)
mapped_full_summary["avg_reading"] = mapped_full_summary["avg_reading"].map("{:.2f}".format)
mapped_full_summary["passing_math_perc"] = mapped_full_summary["passing_math_perc"].map("%{:.2f}".format)
mapped_full_summary["passing_reading_perc"] = mapped_full_summary["passing_reading_perc"].map("%{:.2f}".format)
mapped_full_summary["passing_overall_perc"] = mapped_full_summary["passing_overall_perc"].map("%{:.2f}".format)

#rename columns
clean_full_summary = mapped_full_summary.rename(columns={
    "school_name": "School",
    "type": "Type",
    "size": "Student Population",
    "budget": "School Budget",
    "budget_per_student": "Per Capita Budget",
    "avg_math": "Math Average",
    "avg_reading": "Reading Average",
    "passing_math_perc": "% of Students Passing Math",
    "passing_reading_perc": "% of Students Passing Reading",
    "passing_overall_perc": "% of Students Passing"
})

clean_full_summary


,School,Type,Student Population,School Budget,Per Capita Budget,Math Average,Reading Average,% of Students Passing Math,% of Students Passing Reading,% of Students Passing
0,Bailey High School,District,"4,976","$3,124,928",$628.00,77.05,81.03,%66.68,%81.93,%54.64
1,Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,%94.13,%97.04,%91.33
2,Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,%65.99,%80.74,%53.20
3,Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,%68.31,%79.30,%54.29
4,Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,%93.39,%97.14,%90.60
5,Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,%66.75,%80.86,%53.53
6,Holden High School,Charter,427,"$248,087",$581.00,83.80,83.81,%92.51,%96.25,%89.23
7,Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,%65.68,%81.32,%53.51
8,Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,%66.06,%81.22,%53.54
9,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,%94.59,%95.95,%90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [21]:
#Sort in decending order by overall passing
top_summary = full_summary_df.sort_values("passing_overall_perc", ascending=False).reset_index(drop=True)

#map columns for formatting
top_summary["size"] = top_summary["size"].map("{:,}".format)
top_summary["budget"] = top_summary["budget"].map("${:,}".format)
top_summary["budget_per_student"] = top_summary["budget_per_student"].map("${:.2f}".format)
top_summary["avg_math"] = top_summary["avg_math"].map("{:.2f}".format)
top_summary["avg_reading"] = top_summary["avg_reading"].map("{:.2f}".format)
top_summary["passing_math_perc"] = top_summary["passing_math_perc"].map("%{:.2f}".format)
top_summary["passing_reading_perc"] = top_summary["passing_reading_perc"].map("%{:.2f}".format)
top_summary["passing_overall_perc"] = top_summary["passing_overall_perc"].map("%{:.2f}".format)

#rename columns
clean_top_summary = top_summary.rename(columns={
    "school_name": "School",
    "type": "Type",
    "size": "Student Population",
    "budget": "School Budget",
    "budget_per_student": "Per Capita Budget",
    "avg_math": "Math Average",
    "avg_reading": "Reading Average",
    "passing_math_perc": "% of Students Passing Math",
    "passing_reading_perc": "% of Students Passing Reading",
    "passing_overall_perc": "% of Students Passing"
})
#Take first 5 values only
top_five_summary = clean_top_summary.head()

top_five_summary



,School,Type,Student Population,School Budget,Per Capita Budget,Math Average,Reading Average,% of Students Passing Math,% of Students Passing Reading,% of Students Passing
0,Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,%94.13,%97.04,%91.33
1,Thomas High School,Charter,"1,635","$1,043,130",$638.00,83.42,83.85,%93.27,%97.31,%90.95
2,Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,%93.39,%97.14,%90.60
3,Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.27,83.99,%93.87,%96.54,%90.58
4,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,%94.59,%95.95,%90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [28]:
#Sort in ascending order by overall passing
bottom_summary = full_summary_df.sort_values("passing_overall_perc").reset_index(drop=True)

#map columns for formatting
bottom_summary["size"] = bottom_summary["size"].map("{:,}".format)
bottom_summary["budget"] = bottom_summary["budget"].map("${:,}".format)
bottom_summary["budget_per_student"] = bottom_summary["budget_per_student"].map("${:.2f}".format)
bottom_summary["avg_math"] = bottom_summary["avg_math"].map("{:.2f}".format)
bottom_summary["avg_reading"] = bottom_summary["avg_reading"].map("{:.2f}".format)
bottom_summary["passing_math_perc"] = bottom_summary["passing_math_perc"].map("%{:.2f}".format)
bottom_summary["passing_reading_perc"] = bottom_summary["passing_reading_perc"].map("%{:.2f}".format)
bottom_summary["passing_overall_perc"] = bottom_summary["passing_overall_perc"].map("%{:.2f}".format)

#rename columns
clean_bottom_summary = bottom_summary.rename(columns={
    "school_name": "School",
    "type": "Type",
    "size": "Student Population",
    "budget": "School Budget",
    "budget_per_student": "Per Capita Budget",
    "avg_math": "Math Average",
    "avg_reading": "Reading Average",
    "passing_math_perc": "% of Students Passing Math",
    "passing_reading_perc": "% of Students Passing Reading",
    "passing_overall_perc": "% of Students Passing"
    })
#Take first 5 values only
bottom_five_summary = clean_bottom_summary.head()

bottom_five_summary

,School,Type,Student Population,School Budget,Per Capita Budget,Math Average,Reading Average,% of Students Passing Math,% of Students Passing Reading,% of Students Passing
0,Rodriguez High School,District,"3,999","$2,547,363",$637.00,76.84,80.74,%66.37,%80.22,%52.99
1,Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,%65.99,%80.74,%53.20
2,Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,%65.68,%81.32,%53.51
3,Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,%66.75,%80.86,%53.53
4,Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,%66.06,%81.22,%53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
#Filter the student data data frame by grade
grade_9 = student_data.loc[student_data["grade"] == "9th", :]
grade_10 = student_data.loc[student_data["grade"] == "10th", :]
grade_11 = student_data.loc[student_data["grade"] == "11th", :]
grade_12 = student_data.loc[student_data["grade"] == "12th", :]

#Group by school for each grade
grade_9_grouped = grade_9.groupby(["school_name"]).mean()
grade_10_grouped = grade_10.groupby(["school_name"]).mean()
grade_11_grouped = grade_11.groupby(["school_name"]).mean()
grade_12_grouped = grade_12.groupby(["school_name"]).mean()

#extract math scores
grade_9_math = grade_9_grouped["math_score"]
grade_10_math = grade_10_grouped["math_score"]
grade_11_math = grade_11_grouped["math_score"]
grade_12_math = grade_12_grouped["math_score"]

#Place scores in a data frame
math_avgs_df = pd.DataFrame({"9th": grade_9_math, "10th": grade_10_math, 
                             "11th": grade_11_math, "12th": grade_12_math})

math_avgs_df

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
#Already have a grouped scores data frame above
#extract math scores
grade_9_reading = grade_9_grouped["reading_score"]
grade_10_reading = grade_10_grouped["reading_score"]
grade_11_reading = grade_11_grouped["reading_score"]
grade_12_reading = grade_12_grouped["reading_score"]

#Place scores in a data frame
reading_avgs_df = pd.DataFrame({"9th": grade_9_reading, "10th": grade_10_reading, 
                             "11th": grade_11_reading, "12th": grade_12_reading})

reading_avgs_df

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
#define bins and bin names
budget_bins = [0, 600, 625, 650, 700]
budget_bin_names = ["< 600", "600-625", "625-650", "> 650"]

#new column with bin designation
sorted_full_summary["budget_range"] = pd.cut(sorted_full_summary["budget_per_student"], budget_bins, 
                                            labels= budget_bin_names, include_lowest= True)

sorted_full_summary

scores_by_spending = sorted_full_summary.groupby("budget_range").mean()

#delete unecessary columns
del scores_by_spending["size"]
del scores_by_spending["budget"]
del scores_by_spending["budget_per_student"]

scores_by_spending

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
#define bins and bin names
size_bins = [0, 2000, 3000, 4000, 5000]
size_bin_names = ["< 2000", "2000-3000", "3000-4000", "> 4000"]

#add new column with bin designation
sorted_full_summary["size_range"] = pd.cut(sorted_full_summary["size"], size_bins, 
                                           labels= size_bin_names, include_lowest=True)

#group by bin
scores_by_size= sorted_full_summary.groupby("size_range").mean()

#delete unecessary columns
del scores_by_size["size"]
del scores_by_size["budget"]
del scores_by_size["budget_per_student"]

scores_by_size

## Scores by School Type

* Perform the same operations as above, based on school type

In [ ]:
#group summary table by school type

scores_by_type = sorted_full_summary.groupby("type").mean()

#delete unecessary columns
del scores_by_type["size"]
del scores_by_type["budget"]
del scores_by_type["budget_per_student"]

scores_by_type